In [ ]:
from google.colab import drive
drive.mount("/content/MyDrive/")

Drive already mounted at /content/MyDrive/; to attempt to forcibly remount, call drive.mount("/content/MyDrive/", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df = pd.read_csv("/content/MyDrive/MyDrive/Allfalltypesnew.csv")

In [ ]:
!pip install -q --upgrade wandb

In [ ]:
import pandas as pd
import numpy as np

#Data Visualization
import matplotlib.pyplot as plt

#Text Color
from termcolor import colored

#Train Test Split
from sklearn.model_selection import train_test_split

#Model Evaluation
from sklearn.metrics import confusion_matrix, precision_score, recall_score, accuracy_score
from mlxtend.plotting import plot_confusion_matrix

#Deep Learning
from keras.models import Sequential
from keras.layers import SimpleRNN,Embedding,Dense,LSTM,GRU

In [ ]:
import wandb
from wandb.keras import WandbCallback

# Login to wandb
wandb.init()
config=wandb.config

wandb: Currently logged in as: taariqdb (use `wandb login --relogin` to force relogin)


In [ ]:
config.epochs=3
config.batch_size=60
config.optimizer='adam'

In [ ]:
cols_to_scale = ['AccX','AccY','AccZ','GyrX','GyrY','GyrZ','EulerX','EulerY','EulerZ']

from sklearn.preprocessing import MinMaxScaler 
scaler = MinMaxScaler()
df[cols_to_scale] = scaler.fit_transform(df[cols_to_scale])

In [ ]:
df = pd.get_dummies(data = df, columns=['DescriptionID'])
df = pd.get_dummies(data = df, columns=['FallTypes'])
df.columns

Index(['AccX', 'AccY', 'AccZ', 'GyrX', 'GyrY', 'GyrZ', 'EulerX', 'EulerY',
       'EulerZ', 'TrialID', 'TaskID', 'Subject', 'Description', 'FTLabels',
       'DescriptionID_1', 'DescriptionID_2', 'DescriptionID_3',
       'DescriptionID_4', 'DescriptionID_5', 'DescriptionID_6',
       'DescriptionID_7', 'DescriptionID_8', 'DescriptionID_9',
       'DescriptionID_10', 'DescriptionID_11', 'DescriptionID_12',
       'DescriptionID_13', 'DescriptionID_14', 'DescriptionID_15',
       'FallTypes_Backward', 'FallTypes_Dampen', 'FallTypes_Forward',
       'FallTypes_Lateral'],
      dtype='object')

In [ ]:
X = df[['AccX', 'AccY', 'AccZ', 'GyrX', 'GyrY', 'GyrZ', 'EulerX', 'EulerY','EulerZ',
       'DescriptionID_1', 'DescriptionID_2', 'DescriptionID_3',
       'DescriptionID_4', 'DescriptionID_5', 'DescriptionID_6',
       'DescriptionID_7', 'DescriptionID_8', 'DescriptionID_9',
       'DescriptionID_10', 'DescriptionID_11', 'DescriptionID_12',
       'DescriptionID_13', 'DescriptionID_14', 'DescriptionID_15',]]
X = np.asarray(X)
y = df[['FallTypes_Backward', 'FallTypes_Dampen', 'FallTypes_Forward','FallTypes_Lateral']]
y = np.asarray(y)

In [ ]:
y

array([[0, 0, 1, 0],
       [0, 0, 1, 0],
       [0, 0, 1, 0],
       ...,
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0]], dtype=uint8)

In [ ]:
X.shape, y.shape

((173649, 24), (173649, 4))

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)

In [ ]:
y_test.shape

(34730, 4)

In [ ]:
model1=Sequential()
model1.add(Embedding(10000,24))
model1.add(LSTM(24,activation='sigmoid'))
model1.add(Dense(4,activation='softmax'))

model1.compile(optimizer=config.optimizer,loss='categorical_crossentropy',metrics=['accuracy'])
model1.fit(X_train,y_train,epochs=config.epochs,batch_size=config.batch_size,callbacks=[WandbCallback()])

Epoch 1/3
2316/2316 [==============================] - 249s 105ms/step - loss: 0.3670 - acc: 0.7180 - _timestamp: 1648637392.0000 - _runtime: 267.0000
Epoch 2/3
2316/2316 [==============================] - 277s 119ms/step - loss: 0.1938 - acc: 0.8545 - _timestamp: 1648637668.0000 - _runtime: 543.0000
Epoch 3/3
2316/2316 [==============================] - 225s 97ms/step - loss: 0.0522 - acc: 0.9797 - _timestamp: 1648637893.0000 - _runtime: 768.0000


In [ ]:
#using lstm model
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
labels = ['Backward','Dampen','Forward','Lateral']
pred=model1.predict(X_test)
accuracy=model1.evaluate(X_test,y_test)
print("accuracy is {}".format(accuracy[1]))
print(confusion_matrix(pred.argmax(axis=1),y_test.argmax(axis=1)))
wandb.sklearn.plot_confusion_matrix(y_test.argmax(axis=1), pred.argmax(axis=1),labels)
wandb.termlog('Logged confusion matrix.')
wandb.sklearn.plot_class_proportions(y_train.argmax(axis=1), y_test.argmax(axis=1), ['Forward','Lateral','Backward','Dampen'])
wandb.termlog('Logged class proportions.')

1086/1086 [==============================] - 8s 8ms/step - loss: 0.0151 - acc: 0.9990
accuracy is 0.9989922046661377
[[ 4649     0    10     0]
 [    0  1612     0     0]
 [    0     0 21662    15]
 [    0     2     8  6772]]


wandb: Logged confusion matrix.
wandb: Logged class proportions.
